In [1]:
import pandas as pd
import numpy as np

In [71]:
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
import numpy as np

# load all the DFs
pred_df = pd.read_csv('/home/azstephe/liverRegression/regression_liver/data/model_outputs/mcf297qb/activations_rat_VAL.csv', header=None)
val1_df = pd.read_csv('/home/azstephe/liverRegression/regression_liver/data/splits/val1/rat_liver_VAL.narrowPeak', header=None, delim_whitespace=True).iloc[:,4]
val2_df = pd.read_csv('/home/azstephe/liverRegression/regression_liver/data/splits/val2/rat_liver_VAL.narrowPeak', header=None, delim_whitespace=True)
val3_df = pd.read_csv('/home/azstephe/liverRegression/regression_liver/data/splits/val3/rat_liver_VAL.narrowPeak', header=None, delim_whitespace=True).iloc[:,4]

val1_len = 2*len(val1_df)
val2_len = 2*len(val2_df)
val3_len = 2*len(val3_df)

# input sanity check
if len(pred_df) != val1_len+val2_len+val3_len:
    print("ERROR: predictions are a different length than validation sets")

doubled_val1_df = pd.concat([val1_df, val1_df]).sort_index(kind='mergesort').reset_index(drop=True)
doubled_val2_df = pd.concat([val2_df, val2_df]).sort_index(kind='mergesort').reset_index(drop=True)
doubled_val3_df = pd.concat([val3_df, val3_df]).sort_index(kind='mergesort').reset_index(drop=True)

pred_val1_df = pred_df.head(val1_len)
pred_val2_df = pred_df.iloc[val1_len:val1_len + val2_len]
pred_val3_df = pred_df.tail(val3_len)

doubled_val2_df.rename(columns={0: 'chr'}, inplace=True)

val2 = pd.concat([doubled_val2_df, pred_val2_df.reset_index(drop=True)], axis=1)
val2 = val2.rename(columns={4: 'true', 0: 'pred'})

pred_avg = (val2.loc[::2, 'pred'].values + val2.loc[1::2, 'pred'].values) / 2

val2.loc[::2, 'pred_avg'] = pred_avg

val2_avg = val2.loc[::2]

val2_avg.drop('pred', inplace=True, axis=1)

val2_avg['diff'] = abs(val2_avg['true'] - val2_avg['pred_avg'])

val2_avg = val2_avg.sort_values('diff')
val2_avg

/tmp/ipykernel_11110/3868988573.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val2_avg.drop('pred', inplace=True, axis=1)
/tmp/ipykernel_11110/3868988573.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val2_avg['diff'] = abs(val2_avg['true'] - val2_avg['pred_avg'])


,chr,1,2,3,true,pred_avg,diff
398,chr19,42817113,42817613,peak8769,1.597668,1.591535,0.006133
438,chr19,59405294,59405794,peak8826,1.940120,1.950589,0.010470
306,chr19,1138329,1138829,peak8465,2.478815,2.462476,0.016339
348,chr19,23038836,23039336,peak8620,1.544648,1.564162,0.019514
568,chr8,34153597,34154097,peak19365,2.180264,2.204152,0.023888
...,...,...,...,...,...,...,...
490,chr8,23723729,23724229,peak19220,2.854326,0.859333,1.994993
458,chr8,6119591,6120091,peak19093,3.133687,1.121780,2.011906
678,chr8,86510018,86510518,peak19558,2.629900,0.605840,2.024060
262,chr16,83887691,83888191,peak6774,2.672957,0.551044,2.121914


In [78]:
# val2_avg.tail(150)

val2_avg.tail(150).to_csv('/home/azstephe/liverRegression/regression_liver/data/rat_motif/worst150.csv', index=False, header=False, sep='\t')
val2_avg.head(150).to_csv('/home/azstephe/liverRegression/regression_liver/data/rat_motif/best150.csv', index=False, header=False, sep='\t')

In [79]:
ratMouse = pd.read_csv('/home/azstephe/liverRegression/regression_liver/data/splits/ratMouse/ratToMouse_liver_ratEnhancer_mouseEnhancer_val_wawb.narrowPeak', header=None, delim_whitespace=True)
# ratMouse.merge(val2_avg, how='inner', left_on=3, right_on=3, suffixes=('', '_rat'))
merged = ratMouse.merge(val2_avg, how='inner', left_on=3, right_on=3, suffixes=('', '_rat'))
merged

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,chr,1_rat,2_rat,true,pred_avg,diff
0,chr8,71312546,71313084,peak6185,-1,.,-1,-1,-1,302,...,7.46433,43.51744,40.89712,190,chr16,19703871,19704371,2.165324,0.734635,1.430690
1,chr8,69023471,69023747,peak6207,-1,.,-1,-1,-1,154,...,5.10220,22.43992,20.03460,103,chr16,22411328,22411828,2.234581,2.409515,0.174934
2,chr8,69023471,69023747,peak6207,-1,.,-1,-1,-1,154,...,5.95257,29.55262,27.06180,371,chr16,22411328,22411828,2.234581,2.409515,0.174934
3,chr8,68953917,68954442,peak6210,-1,.,-1,-1,-1,302,...,19.84607,329.66754,326.01117,374,chr16,22493773,22494273,3.006526,1.052040,1.954486
4,chr8,68926297,68927082,peak6211,-1,.,-1,-1,-1,491,...,3.02353,8.02632,5.92528,100,chr16,22514093,22514593,2.616611,1.267980,1.348631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,chr9,117715268,117715626,peak19840,-1,.,-1,-1,-1,198,...,14.83517,173.29382,170.06235,574,chr8,126089777,126090277,2.436796,2.055934,0.380862
582,chr9,118960439,118960843,peak19849,-1,.,-1,-1,-1,229,...,6.04705,30.37843,27.87881,132,chr8,127638531,127639031,2.036453,2.250025,0.213572
583,chr9,120994279,120994581,peak19853,-1,.,-1,-1,-1,141,...,10.39337,74.35537,71.53254,189,chr8,129663774,129664274,2.595143,2.317318,0.277825
584,chr9,122635183,122635840,peak19859,-1,.,-1,-1,-1,136,...,7.27536,41.68927,39.08389,178,chr8,131690999,131691499,2.403540,1.121422,1.282118


In [85]:
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
import numpy as np

both_df = pd.read_csv("/home/azstephe/liverRegression/regression_liver/data/splits/ratMouse/ratToMouse_liver_ratEnhancer_mouseEnhancer_val_wawb.narrowPeak", sep="\t", header=None)
mac_qn = pd.read_csv("/home/azstephe/liverRegression/regression_liver/data/ladder_qn/rat_liver_pos_ALL.bed", sep="\t", header=None)
mouse_qn = pd.read_csv("/home/azstephe/liverRegression/regression_liver/data/ladder_qn/mouse_liver_pos_ALL.bed", sep="\t", header=None)

unique1_2 = both_df[~both_df.duplicated(subset=[1, 2], keep=False)] # entries with unique mac mapped to mouse start and end
all_unique = unique1_2[~unique1_2.duplicated(subset=[11,12], keep=False)]

#### FUNCTIONS

def pearson_spearman(x, y):
    pearson_corr, pearson_p_value = scipy.stats.pearsonr(x, y)
    print(f"Pearson correlation coefficient: {pearson_corr:.4f}, p-value: {pearson_p_value:.4g}")

    spearman_corr, spearman_p_value = scipy.stats.spearmanr(x, y)
    print(f"Spearman correlation coefficient: {spearman_corr:.4f}, p-value: {spearman_p_value:.4g}")
    
def collapse_if_identical(lst):
    return lst[0] if all(x == lst[0] for x in lst) else lst
    

# takes in list and returns peak with largest signal
def keep_largest_signal(lst, qn_df):
    if isinstance(lst, list):
        maxPeakName = ''
        maxPeakStrength = 0
        for peak in lst:
            matching_rows = qn_df[qn_df[3] == peak]
            if not matching_rows.empty:
                signal = matching_rows[4].iloc[0]
                if signal > maxPeakStrength:
                    maxPeakStrength = signal
                    maxPeakName = peak
        return maxPeakName
    else: 
        return lst

# def overlap(start1, end1, start2, end2):
#     return max(0, min(end1, end2)-max(start1, start2))

def get_biggest_overlap(lst, col, df):
    maxOverlap = 0
    maxPeakOverlap = ""
    for peak in lst:
        row = df[df[col] == peak].iloc[0] #iloc bc possible to return multiple rows (never will based on th
        overlap = max(0, min(row[2], row[12])-max(row[1], row[11]))
        if overlap > maxOverlap:
            maxOverlap = overlap
            maxPeakOverlap = peak
    return peak

#### COL1,2 DUPLICATES

duplicates3 = both_df[both_df.duplicated(subset=[1, 2], keep=False)] # entries with duplicated mac mapped to mouse start and end 
grouped_dups3 = duplicates3.groupby([1, 2])[3].apply(list).reset_index(name='col3')

# grouped dups: start | end | [peaks with these endpoints]
grouped_dups3['col3'] = grouped_dups3['col3'].apply(collapse_if_identical) 

grouped_dups3['col3'] = grouped_dups3['col3'].apply(lambda x: keep_largest_signal(x, mac_qn)) # redundant endpoints now map to 1 peak in col3
grouped_dups3 = grouped_dups3[grouped_dups3['col3'] != ''] # remove macaque peaks that aren't large enough (largest 20615)

# df of duplicated col1,2 with strongest peak
keep_strongestcol3 = duplicates3.merge(grouped_dups3[['col3']], left_on=3, right_on='col3', how='inner').drop('col3', axis=1) # keeps the strongest signal in col3 for redundant endpoints

#### COL11,12 DUPLICATES

grouped_dups13 = keep_strongestcol3.groupby([11, 12])[13].apply(list).reset_index(name='col13')
grouped_dups13['col13'] = grouped_dups13['col13'].apply(collapse_if_identical)

grouped_dups13['col13'] = grouped_dups13['col13'].apply(lambda x: keep_largest_signal(x, mouse_qn))
grouped_dups13 = grouped_dups13[grouped_dups13['col13'] != '']

keep_strongestcol13 = keep_strongestcol3.merge(grouped_dups13[['col13']], left_on=13, right_on='col13', how='inner').drop('col13', axis=1)

unique_endpoints = keep_strongestcol13 #rows with unique endpoints from the duplicated endpoints set

####

# col3 peakname duplicates with different endpoints
still_dups_col3 = unique_endpoints[unique_endpoints.duplicated(subset=[3], keep=False)]

# col3peaks | [col13 peaks intersecting col3 peak]
grouped_dcol3 = still_dups_col3.groupby(3)[13].apply(list).reset_index(name='col13')

# get the col13 peak with most overlap of col3
grouped_dcol3['col13'] = grouped_dcol3['col13'].apply(lambda x: get_biggest_overlap(x, 13, still_dups_col3))

merged3 = still_dups_col3.merge(grouped_dcol3[[3, 'col13']], left_on=[3, 13], right_on=[3, 'col13'], how='left', indicator=True)

remove3 = merged3[merged3['_merge'] == 'left_only'].drop(columns=['_merge']) # col13 is what we want to remove

unique_endpoints_subset = unique_endpoints.iloc[:,:20]
remove3_subset = remove3.iloc[:,:20]

# all col3 entries unique
unique3 = unique_endpoints[~unique_endpoints_subset.apply(tuple, axis=1).isin(remove3_subset.apply(tuple, axis=1))] 

####

# col13 peakname duplicates with different endpoints
still_dups_col13 = unique3[unique3.duplicated(subset=[13], keep=False)]

# col13peaks | [col3 peaks intersecting col13 peak]
grouped_dcol13 = still_dups_col13.groupby(13)[3].apply(list).reset_index(name='col3')

# get the col3 peak with most overlap of col13
grouped_dcol13['col3'] = grouped_dcol13['col3'].apply(lambda x: get_biggest_overlap(x, 3, still_dups_col13))

merged13 = still_dups_col13.merge(grouped_dcol13[[13, 'col3']], left_on=[3, 13], right_on=['col3', 13], how='left', indicator=True)

remove13 = merged13[merged13['_merge'] == 'left_only'].drop(columns=['_merge'])

unique3_subset = unique3.iloc[:, :20]
remove13_subset = remove13.iloc[:, :20]

# Identify rows in u3 that are NOT in remove3
filtered_peaks_unique = unique3[~unique3_subset.apply(tuple, axis=1).isin(remove13_subset.apply(tuple, axis=1))]

# one_to_one_peaks = pd.concat(all_unique, filtered_peaks_unique)

full_unique = pd.concat([all_unique, filtered_peaks_unique])

# Sort by column '1'
one_to_one_peaks = full_unique.sort_values(by=1).reset_index(drop=True)
one_to_one_peaks

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,chr8,4868026,4869408,peak6834,-1,.,-1,-1,-1,925,chr8,4868616,4869350,peak19401,1000,.,19.95528,326.98541,323.33551,321
1,chr8,5030308,5030979,peak6832,-1,.,-1,-1,-1,454,chr8,5030444,5031164,peak19405,1000,.,21.38158,606.23029,601.98456,407
2,chr9,6668228,6668942,peak19085,-1,.,-1,-1,-1,393,chr9,6668246,6669123,peak20464,1000,.,10.41801,106.98440,104.00175,372
3,chr8,7060047,7060862,peak6823,-1,.,-1,-1,-1,473,chr8,7060235,7060857,peak19412,1000,.,20.94845,471.64343,467.66739,264
4,chr8,7092461,7093394,peak6821,-1,.,-1,-1,-1,531,chr8,7092243,7093169,peak19415,1000,.,20.20802,313.78287,310.16553,738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,chr8,126807261,126807862,peak8833,-1,.,-1,-1,-1,456,chr8,126807232,126807976,peak20428,921,.,5.94463,33.65896,31.12857,206
394,chr8,127104645,127104941,peak8840,-1,.,-1,-1,-1,160,chr8,127104687,127104941,peak20442,1000,.,8.50367,53.94794,51.25016,130
395,chr8,128025705,128025919,peak8848,-1,.,-1,-1,-1,101,chr8,128024988,128025912,peak20446,604,.,4.91323,20.94289,18.55824,419
396,chr8,128387255,128387673,peak8850,-1,.,-1,-1,-1,229,chr8,128387206,128387634,peak20449,775,.,4.62977,18.75924,16.40690,267


In [100]:
ratTail = val2_avg.tail(150)
ratHead = val2_avg.head(150)

mouseTail = one_to_one_peaks[one_to_one_peaks[3].isin(ratTail[3])][[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]
mouseHead = one_to_one_peaks[one_to_one_peaks[3].isin(ratHead[3])][[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]]

mouseTail.to_csv('/home/azstephe/liverRegression/regression_liver/data/rat_motif/mouseTail.csv', index=False, header=False, sep='\t')
mouseHead.to_csv('/home/azstephe/liverRegression/regression_liver/data/rat_motif/mouseHead.csv', index=False, header=False, sep='\t')


In [99]:
tail

,10,11,12,13,14,15,16,17,18,19
0,chr8,4868616,4869350,peak19401,1000,.,19.95528,326.98541,323.33551,321
1,chr8,5030444,5031164,peak19405,1000,.,21.38158,606.23029,601.98456,407
2,chr9,6668246,6669123,peak20464,1000,.,10.41801,106.98440,104.00175,372
4,chr8,7092243,7093169,peak19415,1000,.,20.20802,313.78287,310.16553,738
5,chr9,7914173,7914971,peak20484,1000,.,15.21739,160.17427,156.98676,531
...,...,...,...,...,...,...,...,...,...,...
381,chr8,120437499,120438014,peak20383,820,.,7.27536,41.68927,39.08389,300
382,chr8,120441674,120442078,peak20384,1000,.,5.85808,28.73361,26.25177,206
385,chr9,122635127,122635473,peak21394,1000,.,7.27536,41.68927,39.08389,178
394,chr8,127104687,127104941,peak20442,1000,.,8.50367,53.94794,51.25016,130
